In [9]:
import sys
sys.path.append("..")
from common.data_loader.load_movie_review import *
from common.lng_processor.eng_process import *
from common.utils.sequence_utils import *
from common.utils.type_cast_utils import *
import torch
import numpy as np

In [ ]:
print("loading and cleaning data")
df = load_movie_review()
df['text_remove_punctuations'] = \
    df['text'].apply(lambda x: remove_punctuations_array(x))
df.info()
# print(df.loc[0:0, ['text']])
# print(df.loc[0:0, ['text_remove_punctuations']])

In [ ]:
print("encoding")
word_dict = build_word_to_int_dict(df['text_remove_punctuations'])
print(word_dict.total_token)
print(word_dict.total_uniq_word)

df['encode'] = df['text_remove_punctuations'].apply(lambda x: word_dict.encode_tokenized_sentence(x))
df['label_encode'] = df['label'].apply(lambda x: 0 if x == 'neg' else 1)

print(df.loc[0:0, ['text_remove_punctuations']])
print(df.loc[0:0, ['encode']])
df.drop(columns=['text_remove_punctuations', 'text', 'label'], inplace=True)

print(df.loc[0:1])

In [ ]:
print("partition samples")
total_sample, _ = df.shape
batch_size = 16
partition_num = total_sample / batch_size

sub_df = np.array_split(df, partition_num)
print(len(sub_df))

In [ ]:
for batch_df in sub_df:
    batch_df['sentence_len'] = batch_df['encode'].apply(lambda x: len(x))
    batch_df.sort_values('sentence_len', ascending=False, inplace=True)
    X = series_to_list(batch_df['encode'])
    Y = series_to_list(batch_df['label_encode'])
    X_length = get_length_of_list_of_list(X)